In [1]:
import pandas as pd 
import numpy as np 

Загрузим данные по ОФЗ. Источник - сайт ММВБ
https://www.moex.com/s2644#/?sec_type%5B%5D='stock_ofz_bond'&currencyid%5B%5D='rub'&faceunit%5B%5D='rub'&bg%5B%5D='stock_tplus','stock_t0','ir_bonds_main_tplus','stock_bonds_d_main_tplus'&coupon_frequency%5B%5D='2'&q=''

In [54]:
bonds = pd.read_csv('rates_bonds.csv', delimiter=';', decimal=',')
bonds.head()

,SECID,SHORTNAME,NAME,TYPENAME,ISIN,REGNUMBER,LISTLEVEL,FACEVALUE,FACEUNIT,ISSUESIZE,...,RTL3,RTH3,DISCOUNT1,LIMIT1,DISCOUNT2,LIMIT2,DISCOUNT3,DISCOUNTL0,DISCOUNTH0,FULLCOVERED
0,SU25085RMFS0,??? 25085,???-?? 25085 24/09/2025,???,RU000A103BQ2,25085RMFS,1,1000.0,RUB,15000000,...,849.69,997.47,6.0,1000000.0,7.0,5000000.0,8.0,5.0,5.0,0
1,SU26207RMFS9,??? 26207,???-?? 26207 03/02/27,???,RU000A0JS3W6,26207RMFS,1,1000.0,RUB,350000000,...,854.35,1023.35,7.0,1000000.0,8.0,5000000.0,9.0,7.0,7.0,0
2,SU26212RMFS9,??? 26212,???-?? 26212 19/01/28,???,RU000A0JTK38,26212RMFS,1,1000.0,RUB,358927588,...,791.46,967.34,8.0,1000000.0,9.0,5000000.0,10.0,5.0,8.0,0
3,SU26218RMFS6,??? 26218,???-?? 26218 17/09/31,???,RU000A0JVW48,26218RMFS,1,1000.0,RUB,250000000,...,736.97,957.21,11.0,1000000.0,12.0,5000000.0,13.0,5.0,11.0,0
4,SU26219RMFS4,??? 26219,???-?? 26219 16/09/26,???,RU000A0JWM07,26219RMFS,1,1000.0,RUB,350000000,...,845.75,1013.05,7.0,1000000.0,8.0,5000000.0,9.0,7.0,7.0,0


In [55]:
bonds.columns

Index(['SECID', 'SHORTNAME', 'NAME', 'TYPENAME', 'ISIN', 'REGNUMBER',
       'LISTLEVEL', 'FACEVALUE', 'FACEUNIT', 'ISSUESIZE', 'IS_COLLATERAL',
       'IS_EXTERNAL', 'PRIMARY_BOARDID', 'PRIMARY_BOARD_TITLE', 'MATDATE',
       'IS_RII', 'INCLUDEDBYMOEX', 'DURATION', 'IS_QUALIFIED_INVESTORS',
       'HIGH_RISK', 'COUPONFREQUENCY', 'EVENINGSESSION', 'MORNINGSESSION',
       'WAPRICE', 'YIELDATWAP', 'COUPONDATE', 'COUPONPERCENT', 'COUPONVALUE',
       'COUPONDAYSPASSED', 'COUPONDAYSREMAIN', 'COUPONLENGTH', 'ISSUEDATE',
       'INITIALFACEVALUE', 'SECSUBTYPE', 'STARTDATEMOEX', 'REPLBOND',
       'DAYSTOREDEMPTION', 'OFFERDATE', 'EMITENTNAME', 'INN', 'LOTSIZE',
       'PRICE', 'PRICE_RUB', 'RTL1', 'RTH1', 'RTL2', 'RTH2', 'RTL3', 'RTH3',
       'DISCOUNT1', 'LIMIT1', 'DISCOUNT2', 'LIMIT2', 'DISCOUNT3', 'DISCOUNTL0',
       'DISCOUNTH0', 'FULLCOVERED'],
      dtype='object')

In [56]:
bonds = bonds[['ISIN', 'COUPONPERCENT', 'MATDATE' , 'DAYSTOREDEMPTION' , 'PRICE']]
bonds.head()

,ISIN,COUPONPERCENT,MATDATE,DAYSTOREDEMPTION,PRICE
0,RU000A103BQ2,6.40,24.09.2025,647,90.92
1,RU000A0JS3W6,8.15,03.02.2027,1144,90.96
2,RU000A0JTK38,7.05,19.01.2028,1494,85.14
3,RU000A0JVW48,8.50,17.09.2031,2831,82.80
4,RU000A0JWM07,7.75,16.09.2026,1004,91.05


# Задача 1 
Записать систему уравнений для процентных ставок для рассматриваемой Компании. В данном случае, под "Компанией" подразмевая Российскую Федерацию


In [57]:
from datetime import datetime
from datetime import date
from datetime import timedelta

In [58]:
bonds['MATDATE'] = bonds['MATDATE'].apply(lambda x: datetime.strptime(x, '%d.%m.%Y').date())
bonds.head()

,ISIN,COUPONPERCENT,MATDATE,DAYSTOREDEMPTION,PRICE
0,RU000A103BQ2,6.40,2025-09-24,647,90.92
1,RU000A0JS3W6,8.15,2027-02-03,1144,90.96
2,RU000A0JTK38,7.05,2028-01-19,1494,85.14
3,RU000A0JVW48,8.50,2031-09-17,2831,82.80
4,RU000A0JWM07,7.75,2026-09-16,1004,91.05


Запишем уравнения для поиска процентных ставок в узловых точках

In [71]:
today_ = date(2023, 12, 17)

for index, row in bonds.iterrows():
    p_i = row['PRICE']
    T_i = round(row['DAYSTOREDEMPTION']/365, 2)
    maturity_date = row['MATDATE']
    coupon_days = []
    
    iter_date = maturity_date
    while iter_date > today_:
        if iter_date == maturity_date:
            pass
        else:
            coupon_days.append((iter_date-today_).days)
        iter_date += timedelta(days=-182)
        
    d_coupons_string = ['+ ' + str(row['COUPONPERCENT'])+' * exp(-r(' + str(round(x/365, 2)) + ')*' + str(round(x/365, 2)) + ')' for x in coupon_days]
    print(f"({index+1}) {p_i} = 100 * exp(-r({T_i})*{T_i})", ''.join([str(lst) for lst in  d_coupons_string]))
    #print(d_coupons_string)

(1) 90.92 = 100 * exp(-r(1.77)*1.77) + 6.4 * exp(-r(1.27)*1.27)+ 6.4 * exp(-r(0.78)*0.78)+ 6.4 * exp(-r(0.28)*0.28)
(2) 90.96 = 100 * exp(-r(3.13)*3.13) + 8.15 * exp(-r(2.64)*2.64)+ 8.15 * exp(-r(2.14)*2.14)+ 8.15 * exp(-r(1.64)*1.64)+ 8.15 * exp(-r(1.14)*1.14)+ 8.15 * exp(-r(0.64)*0.64)+ 8.15 * exp(-r(0.14)*0.14)
(3) 85.14 = 100 * exp(-r(4.09)*4.09) + 7.05 * exp(-r(3.59)*3.59)+ 7.05 * exp(-r(3.1)*3.1)+ 7.05 * exp(-r(2.6)*2.6)+ 7.05 * exp(-r(2.1)*2.1)+ 7.05 * exp(-r(1.6)*1.6)+ 7.05 * exp(-r(1.1)*1.1)+ 7.05 * exp(-r(0.6)*0.6)+ 7.05 * exp(-r(0.1)*0.1)
(4) 82.8 = 100 * exp(-r(7.76)*7.76) + 8.5 * exp(-r(7.26)*7.26)+ 8.5 * exp(-r(6.76)*6.76)+ 8.5 * exp(-r(6.26)*6.26)+ 8.5 * exp(-r(5.76)*5.76)+ 8.5 * exp(-r(5.26)*5.26)+ 8.5 * exp(-r(4.76)*4.76)+ 8.5 * exp(-r(4.27)*4.27)+ 8.5 * exp(-r(3.77)*3.77)+ 8.5 * exp(-r(3.27)*3.27)+ 8.5 * exp(-r(2.77)*2.77)+ 8.5 * exp(-r(2.27)*2.27)+ 8.5 * exp(-r(1.77)*1.77)+ 8.5 * exp(-r(1.27)*1.27)+ 8.5 * exp(-r(0.78)*0.78)+ 8.5 * exp(-r(0.28)*0.28)
(5) 91.05 = 100 *

Как применить bootstrap и найти кусочно-постоянное решение - непонятно, так как безкупонных облигаций нет, а близких к погашению (где купоны уже выплачены) - мало, даты выплат не совпадают. Неизбежно приходится делать предположения о связи ставок для разных промежутков времени, а как делать блочный bootstrap в теории не объяснется 


Если бы я не ознакомился с теорией, то решал бы задачу так. Сначала найдем дюрацию и доходность к погашению (yeild to maturity). Эта доходность и есть доходнсть безкупонной облигации с погашением в дату сегодня + дюрация. Иначе, был бы арбитраж между купонной и безкупонной облигацией. Таким образом, мы нашли бы значения кривой ставок в узловых точках - в точках сегодня+дюрация облигаций. Далее нужно было бы искать гладкую кривую, проходящую через эти точки, с найденными значениями доходности. И здесь применять сплайны разных видов 